## Data Extraction, Transformation & Load

In [3]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import duckdb
import warnings
import os

load_dotenv()
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [4]:
motherduck_token = os.getenv("motherduck_token")

In [5]:
conn = duckdb.connect(f'md:NaijaYield?motherduck_token={motherduck_token}')

In [6]:
query = """
CREATE TABLE naijayield_users (
    user_id VARCHAR(36) PRIMARY KEY,  -- Using UUID format
    email VARCHAR(255) NOT NULL UNIQUE,
    name VARCHAR(255) NOT NULL,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    login_count INT DEFAULT 0,
    last_login TIMESTAMP,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
conn.execute(query)
print("user table created")

user table created


In [12]:
data_path = "../data/NGA_2015_GHSP-W3_v02_M_CSV"

In [170]:
path_to_parquet = "../transformed_data"

def create_table_from_parquet(table_name, table_comment, conn=conn):
        query = f"""
                CREATE OR REPLACE TABLE {table_name} AS 
                SELECT * FROM read_parquet('{path_to_parquet}/{table_name}.parquet')
                """
        
        table_comment_sql = f"""
                        COMMENT ON TABLE {table_name} IS '{table_comment}'
                        """
        conn.execute(query)
        conn.execute(table_comment_sql)
        print("=== Table Created ===")


In [69]:
def read_data(file_name):
    if file_name[-3:] != 'csv':
        file_name += '.csv'
    return pd.read_csv(os.path.join(data_path, file_name))

In [116]:
def create_unique_id(df):
    missing_columns = []
    for col in ['hhid', 'indiv']:
        if col not in df.columns:
            missing_columns.append(col)
    
    if missing_columns:
        raise Exception(f"The following required columns are missing: {', '.join(missing_columns)}")
    
    df['UniqueId'] = df['hhid'].astype(str) + '_' + df['indiv'].astype(str)
    return df

In [117]:
sections = [
    "sect3_plantingw3",
    "sect4a_plantingw3",
    "sect4c1_plantingw3",
    "sect4c2_plantingw3",
    "sect4c3_plantingw3",
    "sect11e_plantingw3",
    "sect11f_plantingw3",
    "sect11i_plantingw3",
    "sect11j_plantingw3",
    "sect11k_plantingw3",
    "sectc2a_plantingw3",
    "sectc2b_plantingw3",
    "secta1_harvestw3",
    "secta2_harvestw3",
    "secta3i_harvestw3",
    "secta3ii_harvestw3",
    "secta4_harvestw3",
    "secta8_harvestw3",
    "secta9a1_harvestw3",
    "secta9a2_harvestw3",
    "secta9b1_harvestw3",
    "secta9b2_harvestw3",
    "secta9b3_harvestw3",
    "secta11c2_harvestw3"
]

N.B: In the responses, 1 indicates **YES** while 2 indicates **NO**

In [171]:
df = read_data(sections[0])

In [172]:
df.head()

,zone,state,lga,sector,ea,hhid,indiv,s3q1,s3q2,s3q3,s3q4,s3q4b,s3q5,s3q5b,s3q5c,s3q6,s3q6b,s3q6c,s3q7,s3q8,s3q9,s3q9b,s3q10,s3q11,s3q11b,s3q12b1,s3q13a,s3q13b,s3q14,s3q14b,s3q15,s3q15b,s3q15b1,s3q15c,s3q15d,s3q15e,s3q15f,s3q16,s3q17,s3q18,s3q21a,s3q21b,s3q22a,s3q22b,s3q23,s3q24a,s3q24b,s3q25,s3q26a,s3q26b,s3q27,s3q28,s3q28b,s3q28b1,s3q28c,s3q28d,s3q28e,s3q28f,s3q29,s3q30,s3q31,s3q34a,s3q34b,s3q35a,s3q35b,s3q36,s3q37a,s3q37b,s3q38,s3q38a,s3q39_new,s3q39a_new,s3q40_new
0,4,1,115,1,670,10001,1,1,2.0,2.0,1.0,10.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CIVIL SERVANT,3442.0,13.0,NaN,2.0,NaN,2.0,4.0,2.0,2.0,2.0,11.0,48.0,42.0,67800.0,5.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN
1,4,1,115,1,670,10001,2,1,1.0,NaN,1.0,10.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CIVIL SERVANT,3443.0,13.0,NaN,2.0,NaN,2.0,4.0,2.0,2.0,2.0,11.0,48.0,42.0,42300.0,5.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN
2,4,1,115,1,670,10001,3,1,2.0,2.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN
3,4,1,115,1,670,10001,4,1,1.0,NaN,1.0,15.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NANNY,5139.0,10.0,NaN,5.0,NaN,2.0,1.0,2.0,2.0,2.0,12.0,52.0,68.0,12000.0,5.0,4.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN
4,4,1,115,1,670,10001,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
df = create_unique_id(df)

In [174]:
df

,zone,state,lga,sector,ea,hhid,indiv,s3q1,s3q2,s3q3,s3q4,s3q4b,s3q5,s3q5b,s3q5c,s3q6,s3q6b,s3q6c,s3q7,s3q8,s3q9,s3q9b,s3q10,s3q11,s3q11b,s3q12b1,s3q13a,s3q13b,s3q14,s3q14b,s3q15,s3q15b,s3q15b1,s3q15c,s3q15d,s3q15e,s3q15f,s3q16,s3q17,s3q18,s3q21a,s3q21b,s3q22a,s3q22b,s3q23,s3q24a,s3q24b,s3q25,s3q26a,s3q26b,s3q27,s3q28,s3q28b,s3q28b1,s3q28c,s3q28d,s3q28e,s3q28f,s3q29,s3q30,s3q31,s3q34a,s3q34b,s3q35a,s3q35b,s3q36,s3q37a,s3q37b,s3q38,s3q38a,s3q39_new,s3q39a_new,s3q40_new,UniqueId
0,4,1,115,1,670,10001,1,1,2.0,2.0,1.0,10.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CIVIL SERVANT,3442.0,13.0,NaN,2.0,NaN,2.0,4.0,2.0,2.0,2.0,11.0,48.0,42.0,67800.0,5.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN,10001_1
1,4,1,115,1,670,10001,2,1,1.0,NaN,1.0,10.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CIVIL SERVANT,3443.0,13.0,NaN,2.0,NaN,2.0,4.0,2.0,2.0,2.0,11.0,48.0,42.0,42300.0,5.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN,10001_2
2,4,1,115,1,670,10001,3,1,2.0,2.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN,10001_3
3,4,1,115,1,670,10001,4,1,1.0,NaN,1.0,15.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NANNY,5139.0,10.0,NaN,5.0,NaN,2.0,1.0,2.0,2.0,2.0,12.0,52.0,68.0,12000.0,5.0,4.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN,10001_4
4,4,1,115,1,670,10001,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26730,1,37,3706,2,6,370040,2,1,2.0,1.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,1.0,370040_2
26731,1,37,3706,2,6,370040,3,1,2.0,1.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,1.0,370040_3
26732,1,37,3706,2,6,370040,4,1,2.0,1.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,1.0,370040_4
26733,1,37,3706,2,6,370040,5,1,2.0,1.0,2.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,2.0,NaN,370040_5


In [175]:
selected_vars = {
    "UniqueId":"UniqueId",
    "hhid" : "HouseHoldID",
    "zone": "Region",
    "state": "State",
    "lga": "LocalGovernmentArea",
    "sector": "UrbanRuralSector",
    "s3q13a" : "MainJob",
    "s3q12b1" : "WorkLast7days",
    "s3q21a" : "LastSalary"
}

In [176]:
df = df[list(selected_vars.keys())]
df.rename(columns=selected_vars, inplace=True)

In [177]:
df.to_parquet("../transformed_data/Individual_level_data.parquet", index=False)
create_table_from_parquet("Individual_level_data", sections[0])

=== Table Created ===


___

### savings & insurance :

In [88]:
sections[1]

'sect4a_plantingw3'

In [178]:
df_1 = read_data(sections[1])
df_1 = create_unique_id(df_1)

In [153]:
df_1 

,zone,state,lga,sector,ea,hhid,indiv,s4aq1a,s4aq1,s4aq2a,s4aq2b,s4aq2c,s4aq3,s4aq4,s4aq5,s4aq6,s4aq7,s4aq8,s4aq9b,s4aq9d,s4aq9f,s4aq10,s4aq16,s4aq17a,s4aq17b,s4aq17b_os,s4aq17c,s4aq17d,s4aq17e,s4aq17f,UniqueId
0,4,1,115,1,670,10001,1,1,1.0,1.0,NaN,NaN,2.0,2.0,2.0,NaN,1.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_1
1,4,1,115,1,670,10001,2,1,1.0,8.0,NaN,NaN,2.0,2.0,2.0,NaN,1.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_2
2,4,1,115,1,670,10001,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_3
3,4,1,115,1,670,10001,4,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_4
4,4,1,115,1,670,10001,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10001_8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26866,1,37,3706,2,6,370040,2,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370040_2
26867,1,37,3706,2,6,370040,3,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370040_3
26868,1,37,3706,2,6,370040,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370040_4
26869,1,37,3706,2,6,370040,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370040_5


- Building a farmer finance profile that assesses creditworthiness
- Understanding existing financial behaviors and access points
- Identifying regional patterns in financial inclusion
- Measuring financial literacy and risk management practices
- Developing a credit scoring model that considers formal and informal financial activities

In [179]:
selected_variables = {
    "UniqueId":"UniqueId",
    "hhid" : "HouseHoldID",
    "s4aq1a": "IsAdult",
    "s4aq1": "HasBankAccount",
    "s4aq3": "SoughtAccountInfo",
    "s4aq4": "ConsideredAlternatives",
    "s4aq5": "CheckedDetailedTerms",
    "s4aq6": "ThoroughnessOfTermsReview",
    "s4aq7": "HasProxyBankingAccess",
    "s4aq8": "UsedCooperative",
    "s4aq9b": "SavingsInstitutionType1",
    "s4aq9d": "SavingsInstitutionType2",
    "s4aq9f": "SavingsInstitutionType3",
    "s4aq10": "UsedInformalSavingsGroups",
    "s4aq16": "HasInsurance",
    "s4aq17b": "InsuranceInstitutionType1",
    "s4aq17d": "InsuranceInstitutionType2",
    "s4aq17f": "InsuranceInstitutionType3"
}

In [180]:
df_1 = df_1[list(selected_variables.keys())]
df_1.rename(columns=selected_variables, inplace=True)

In [181]:
df_1.isnull().sum()/len(df_1) < 0.95


UniqueId                      True
HouseHoldID                   True
IsAdult                       True
HasBankAccount                True
SoughtAccountInfo             True
ConsideredAlternatives        True
CheckedDetailedTerms          True
ThoroughnessOfTermsReview     True
HasProxyBankingAccess         True
UsedCooperative               True
SavingsInstitutionType1      False
SavingsInstitutionType2      False
SavingsInstitutionType3      False
UsedInformalSavingsGroups     True
HasInsurance                  True
InsuranceInstitutionType1    False
InsuranceInstitutionType2    False
InsuranceInstitutionType3    False
dtype: bool

In [182]:
valid_columns = (df_1.isnull().sum() / len(df_1)) < 0.95

df_filtered = df_1.loc[:, valid_columns]


In [183]:
df_filtered

,UniqueId,HouseHoldID,IsAdult,HasBankAccount,SoughtAccountInfo,ConsideredAlternatives,CheckedDetailedTerms,ThoroughnessOfTermsReview,HasProxyBankingAccess,UsedCooperative,UsedInformalSavingsGroups,HasInsurance
0,10001_1,10001,1,1.0,2.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0
1,10001_2,10001,1,1.0,2.0,2.0,2.0,NaN,1.0,2.0,2.0,2.0
2,10001_3,10001,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001_4,10001,1,2.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
4,10001_8,10001,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26866,370040_2,370040,1,2.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
26867,370040_3,370040,1,2.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
26868,370040_4,370040,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26869,370040_5,370040,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
df_filtered.to_parquet("../transformed_data/savings_and_insurance_data.parquet", index=False)
create_table_from_parquet("savings_and_insurance_data", sections[1])

=== Table Created ===


### Credit history and loans:


In [132]:
sections[2:]

['sect4c1_plantingw3',
 'sect4c2_plantingw3',
 'sect4c3_plantingw3',
 'sect11e_plantingw3',
 'sect11f_plantingw3',
 'sect11i_plantingw3',
 'sect11j_plantingw3',
 'sect11k_plantingw3',
 'sectc2a_plantingw3',
 'sectc2b_plantingw3',
 'secta1_harvestw3',
 'secta2_harvestw3',
 'secta3i_harvestw3',
 'secta3ii_harvestw3',
 'secta4_harvestw3',
 'secta8_harvestw3',
 'secta9a1_harvestw3',
 'secta9a2_harvestw3',
 'secta9b1_harvestw3',
 'secta9b2_harvestw3',
 'secta9b3_harvestw3',
 'secta11c2_harvestw3']

the data on Credit history and loans all lack indiv, so this will be treated on a household basis

In [185]:
def subset_rename_data(df, selected_variables):
    df = df[list(selected_variables.keys())]
    df = df.rename(columns=selected_variables)
    return df

In [186]:
df_2 = read_data(sections[2])

df_3 = read_data(sections[3])

df_4 = read_data(sections[4])


In [187]:
df_2.head()

,zone,state,lga,sector,ea,hhid,s4cq1
0,4.0,1.0,115.0,1.0,670.0,10001,2.0
1,4.0,1.0,115.0,1.0,670.0,10002,2.0
2,4.0,1.0,115.0,1.0,670.0,10003,2.0
3,4.0,1.0,115.0,1.0,670.0,10004,2.0
4,4.0,1.0,115.0,1.0,670.0,10005,1.0


In [188]:
selected_variables = {
    "hhid" : "HouseHoldID",
    "s4cq1" : "Borrowed_Or_appliedLoan"
}

In [189]:
df_2 = subset_rename_data(df_2, selected_variables)

In [190]:
df_2.to_parquet("../transformed_data/credit_history_loan_1.parquet", index=False)
create_table_from_parquet("credit_history_loan_1", sections[2])

=== Table Created ===


In [191]:
df_3 #sect4c2_plantingw3

,zone,state,lga,sector,ea,hhid,lid,s4cq2b,s4cq3a,s4cq3b,s4cq3c,s4cq4,s4cq4_os,s4cq5,s4cq6,s4cq7,s4cq8,s4cq8b,s4cq9,s4cq10,s4cq10b,s4cq11
0,4,1,115,1,670,10005,1,4.0,1.0,NaN,NaN,9,NaN,1,1000000.0,1.0,2.0,2015.0,2.0,3.0,2018.0,1280000.0
1,4,1,115,1,670,10005,2,6.0,1.0,NaN,NaN,7,NaN,1,20000.0,1.0,9.0,2015.0,2.0,9.0,2015.0,20000.0
2,4,1,106,2,880,10021,1,5.0,1.0,2.0,NaN,2,NaN,1,20000.0,2.0,1.0,2015.0,2.0,12.0,2015.0,20000.0
3,4,1,106,2,880,10021,2,6.0,2.0,NaN,NaN,2,NaN,1,5000.0,2.0,3.0,2015.0,2.0,12.0,2015.0,5000.0
4,4,1,106,2,880,10022,1,1.0,1.0,NaN,NaN,7,NaN,1,50000.0,1.0,9.0,2014.0,2.0,9.0,2015.0,70000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,6,30,3014,1,0,370021,2,1.0,1.0,NaN,NaN,10,NaN,1,100000.0,1.0,8.0,2015.0,2.0,7.0,2016.0,105000.0
896,1,37,3702,1,972,370023,1,1.0,2.0,NaN,NaN,10,NaN,1,200000.0,2.0,10.0,2014.0,2.0,12.0,2015.0,220000.0
897,1,37,3702,1,972,370025,1,1.0,2.0,NaN,NaN,10,NaN,1,150000.0,2.0,7.0,2015.0,2.0,7.0,2016.0,165000.0
898,1,26,2609,1,0,370036,1,4.0,1.0,NaN,NaN,8,NaN,1,900000.0,1.0,10.0,2014.0,2.0,10.0,2017.0,1300000.0


In [192]:
loan_history_variables = {
    "hhid": "HouseholdID",
    "lid": "LoanID",
    "s4cq2b": "LenderType",
    "s4cq3a": "PrimaryLoanResponsible",
    "s4cq3b": "SecondaryLoanResponsible",
    "s4cq3c": "TertiaryLoanResponsible",
    "s4cq4": "LoanPurpose",
    "s4cq4_os": "LoanPurposeOther",
    "s4cq5": "LoanStatus",
    "s4cq6": "LoanAmount",
    "s4cq7": "LoanSufficient",
    "s4cq8": "LoanReceiveMonth",
    "s4cq8b": "LoanReceiveYear",
    "s4cq9": "IsFullyRepaid",
    "s4cq10": "ExpectedFinalPaymentMonth",
    "s4cq10b": "ExpectedFinalPaymentYear",
    "s4cq11": "TotalAmountPaid"
}

In [193]:
df_3 = subset_rename_data(df_3, loan_history_variables)

In [194]:
df_3.to_parquet("../transformed_data/credit_history_loan_2.parquet", index=False)
create_table_from_parquet("credit_history_loan_2", sections[3])

=== Table Created ===


In [196]:
sections[4]

'sect4c3_plantingw3'

In [195]:
df_4 

,zone,state,lga,sector,ea,hhid,s4cq12,s4cq13,s4cq13_os,s4cq14,s4cq14_os,s4cq14b,s4cq14b_os,s4cq15,s4cq15_os,s4cq15b,s4cq15b_os,s4cq16,s4cq17,s4cq17_os,s4cq17b,s4cq17b_os
0,4,1,115,1,670,10001,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,NaN,3.0,NaN
1,4,1,115,1,670,10002,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
2,4,1,115,1,670,10003,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
3,4,1,115,1,670,10004,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,4,1,115,1,670,10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4606,1,26,2609,1,0,370036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4607,1,37,3706,2,6,370037,1.0,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4608,1,37,3706,2,6,370038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4609,1,37,3706,2,6,370039,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN


In [198]:
loan_rejection_variables = {
    "hhid": "HouseholdID",
    "s4cq12": "LoanApplicationRejected",
    "s4cq13": "RejectedLoanPurpose",
    "s4cq13_os": "RejectedLoanPurposeOther",
    "s4cq14": "PrimaryRejectionSource",
    "s4cq14_os": "PrimaryRejectionSourceOther",
    "s4cq14b": "SecondaryRejectionSource",
    "s4cq15": "PrimaryRejectionReason",
    "s4cq15_os": "PrimaryRejectionReasonOther",
    "s4cq15b": "SecondaryRejectionReason",
    "s4cq16": "NeededLoan",
    "s4cq17": "PrimaryReasonNoBorrowing",
    "s4cq17_os": "PrimaryReasonNoBorrowingOther",
    "s4cq17b": "SecondaryReasonNoBorrowing"
}

In [200]:
df_4 = subset_rename_data(df_4, loan_rejection_variables)

In [201]:
df_4.to_parquet("../transformed_data/credit_history_loan_3.parquet", index=False)
create_table_from_parquet("credit_history_loan_3", sections[4])

=== Table Created ===


___

In [203]:
sections[5:]

['sect11e_plantingw3',
 'sect11f_plantingw3',
 'sect11i_plantingw3',
 'sect11j_plantingw3',
 'sect11k_plantingw3',
 'sectc2a_plantingw3',
 'sectc2b_plantingw3',
 'secta1_harvestw3',
 'secta2_harvestw3',
 'secta3i_harvestw3',
 'secta3ii_harvestw3',
 'secta4_harvestw3',
 'secta8_harvestw3',
 'secta9a1_harvestw3',
 'secta9a2_harvestw3',
 'secta9b1_harvestw3',
 'secta9b2_harvestw3',
 'secta9b3_harvestw3',
 'secta11c2_harvestw3']

###  Quantity and Value of Field Crops Produced

In [ ]:
# secta3i_harvestw3
# secta3ii_harvestw3


In [204]:
df_5 = read_data("secta3i_harvestw3")

In [205]:
crop_harvest_variables = {
    "hhid": "HouseholdID",
    "plotid": "PlotID", 
    "cropid": "CropID",
    "cropname": "CropName",
    "cropcode": "CropCode",
    "zone": "Region",
    "state": "State",
    "lga": "LocalGovernmentArea",
    "sector": "UrbanRuralSector",
    "sa3iq3": "CropHarvested",
    "sa3iq4": "ReasonNotHarvested",
    "sa3iq6i": "HarvestQuantity",
    "sa3iq6ii": "HarvestUnit",
    "sa3iq6a": "HarvestValue",
    "sa3iq6b": "HarvestCompleted",
    "sa3iq6d1": "ExpectedAdditionalHarvest",
    "sa3iq6d2": "ExpectedHarvestUnit",
    "sa3iq6e1": "PrimaryHarvestDecisionMaker",
    "sa3iq6e2": "SecondaryHarvestDecisionMaker"
}

In [206]:
df_5 = subset_rename_data(df_5, crop_harvest_variables)

In [207]:
df_5.to_parquet("../transformed_data/crop_harvest_1.parquet", index=False)
create_table_from_parquet("crop_harvest_1", "secta3i_harvestw3")

=== Table Created ===


In [208]:
df_6 = read_data("secta3ii_harvestw3")

In [209]:
df_6.head()

,zone,state,lga,sector,ea,hhid,crop_number,cropname,cropcode,sa3iiq3,sa3iiq4,sa3iiq5a,sa3iiq5b,sa3iiq5b_os,sa3iiq6,sa3iiq7a,sa3iiq7b,sa3iiq8a,sa3iiq8b,sa3iiq9a,sa3iiq9b,sa3iiq10,sa3iiq11a,sa3iiq11b,sa3iiq11b_os,sa3iiq12a,sa3iiq12b,sa3iiq13a,sa3iiq13b,sa3iiq13b_os,sa3iiq14a,sa3iiq14b,sa3iiq14b_os,sa3iiq15a,sa3iiq15b,sa3iiq16a,sa3iiq16b,sa3iiq16b_os,sa3iiq17a,sa3iiq17b,sa3iiq17b_os,sa3iiq18a,sa3iiq18b,sa3iiq18b_os,sa3iiq18c,sa3iiq19,sa3iiq20a,sa3iiq20b,sa3iiq20b_os,sa3iiq21,sa3iiq22a,sa3iiq22b,sa3iiq23a,sa3iiq23b,sa3iiq24,sa3iiq25,sa3iiq25_os,sa3iiq26,sa3iiq27
0,4,1,105,2,0,10009,1,MAIZE,1080,1.0,1.0,2.0,131.0,NaN,2000.0,N02,NaN,2.0,NaN,2.0,NaN,1.0,0.5,131.0,NaN,0.0,NaN,1.5,131.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,1,105,2,0,10009,2,POTATO,2180,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,1.0,140.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,1,105,2,0,10009,3,PLANTAIN,2170,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,2.0,101.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,104,2,1364,10011,1,CASSAVA,1020,1.0,1.0,2.0,131.0,NaN,2000.0,N01,NaN,1.0,NaN,1.0,NaN,1.0,0.0,NaN,NaN,0.0,NaN,2.0,131.0,NaN,0.0,NaN,NaN,1.0,131.0,0.0,NaN,NaN,1.0,131.0,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,104,2,1364,10011,2,MAIZE,1080,1.0,1.0,2.0,131.0,NaN,2000.0,N01,NaN,1.0,NaN,1.0,NaN,1.0,1.0,130.0,NaN,0.0,NaN,1.0,130.0,NaN,0.0,NaN,NaN,1.0,130.0,0.0,NaN,NaN,1.0,130.0,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
crop_disposition_variables = {
    "hhid": "HouseholdID",
    "cropname": "CropName",
    "cropcode": "CropCode",
    "zone": "Region",
    "state": "State",
    "sector": "UrbanRuralSector",
    "sa3iiq3": "SoldUnprocessedCrop",
    "sa3iiq5a": "QuantitySold",
    "sa3iiq5b": "QuantitySoldUnit",
    "sa3iiq6": "SalesValue",
    "sa3iiq7a": "PrimaryBuyer",
    "sa3iiq9a": "PrimaryEarningsDecisionMaker",
    "sa3iiq10": "PaymentPromptness",
    "sa3iiq19": "SoldProcessedCrop",
    "sa3iiq20a": "ProcessedQuantitySold", 
    "sa3iiq21": "ProcessedSalesValue",
    "sa3iiq25": "ProcessingType"
}

In [212]:
df_6 = subset_rename_data(df_6, crop_disposition_variables)
df_6.to_parquet("../transformed_data/crop_harvest_2.parquet", index=False)
create_table_from_parquet("crop_harvest_2", "secta3ii_harvestw3")

=== Table Created ===


###  Income from Sale of Agricultural Products

In [213]:
df_7 = read_data("secta8_harvestw3")

In [214]:
agricultural_byproducts_variables = {
    "hhid": "HouseholdID",
    "zone": "Region",
    "state": "State",
    "sector": "UrbanRuralSector",
    "byprod_cd": "ByProductCode",
    "byprod_desc": "ByProductDescription",
    "sa8q1": "ProducedByProduct",
    "sa8q2": "ProductionMonths",
    "sa8q4": "SoldByProduct",
    "sa8q5a": "QuantitySold",
    "sa8q6": "SalesValue",
    "sa8q8a": "PrimaryEarningsDecisionMaker"
}

In [215]:
df_7 = subset_rename_data(df_7, agricultural_byproducts_variables)
df_7.to_parquet("../transformed_data/agricultural_byproducts.parquet", index=False)
create_table_from_parquet("agricultural_byproducts", "secta8_harvestw3")

=== Table Created ===
